In [1]:
import pandas as pd
import gzip

In [2]:
# List of datasets to use
data = ['Resources/data/reviews_Digital_Music_5.json.gz', 'Resources/data/reviews_Digital_Music_5.json.gz']

# functions to read Amazon data into a pandas data frame
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

# function to concatenate multiple Amazon datasets
def concatDF(data):
    df = pd.DataFrame()
    for dataset in data:
        dftemp = getDF(dataset)
        df = pd.concat([df, dftemp], axis=0)
    # drop unneeded columns
    df.drop(columns = ['reviewerID', 'asin', 'reviewerName', 'helpful', \
                       'summary', 'unixReviewTime', 'reviewTime'], inplace= True)
    return df

In [33]:
df = concatDF(data)

In [34]:
df.head()

,reviewText,overall
0,"It's hard to believe ""Memory of Trees"" came ou...",5.0
1,"A clasically-styled and introverted album, Mem...",5.0
2,I never thought Enya would reach the sublime h...,5.0
3,This is the third review of an irish album I w...,5.0
4,"Enya, despite being a successful recording art...",4.0


In [35]:
df['reviewText'] = df['reviewText'].str.lower()
df = df.iloc[:1000]

In [36]:
df.head()

,reviewText,overall
0,"it's hard to believe ""memory of trees"" came ou...",5.0
1,"a clasically-styled and introverted album, mem...",5.0
2,i never thought enya would reach the sublime h...,5.0
3,this is the third review of an irish album i w...,5.0
4,"enya, despite being a successful recording art...",4.0


In [37]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english")) 

def remove_stops(row):
    list_words = row.split(' ')
    meaningful_words = [word for word in list_words if word.isalpha() and (not word in stops)]
    return_string = ""
    for word in meaningful_words:
        return_string += word + " "  
    return return_string

In [38]:
df['train_me'] = df['reviewText'].apply(remove_stops)

In [39]:
df.head()

,reviewText,overall,train_me
0,"it's hard to believe ""memory of trees"" came ou...",5.0,hard believe came years held well passage last...
1,"a clasically-styled and introverted album, mem...",5.0,introverted memory trees masterpiece many song...
2,i never thought enya would reach the sublime h...,5.0,never thought enya would reach sublime heights...
3,this is the third review of an irish album i w...,5.0,third review irish album write today others su...
4,"enya, despite being a successful recording art...",4.0,despite successful recording broad appeal one ...


In [40]:
x = df['train_me']
y = df['overall'].astype(int)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [42]:
vectorized_train = vectorizer.fit_transform(X_train).toarray()
vectorized_test = vectorizer.transform(X_test).toarray()

In [43]:
from sklearn.preprocessing import Normalizer
norm = Normalizer().fit(vectorized_train)

norm_vectorized_train = norm.transform(vectorized_train)
norm_vectorized_test = norm.transform(vectorized_test)

In [44]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=vectorized_train.shape[1]))
model.add(Dense(units=5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                552704    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 553,029
Trainable params: 553,029
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(norm_vectorized_train, y_train_categorical, epochs=100, validation_data=(norm_vectorized_test, y_test_categorical))

Epoch 1/100
25/25 [==============================] - 0s 10ms/step - loss: 0.2742 - accuracy: 0.9325 - val_loss: 0.8593 - val_accuracy: 0.7550
Epoch 2/100
25/25 [==============================] - 0s 9ms/step - loss: 0.2273 - accuracy: 0.9488 - val_loss: 0.8751 - val_accuracy: 0.7600
Epoch 3/100
25/25 [==============================] - 0s 9ms/step - loss: 0.1864 - accuracy: 0.9588 - val_loss: 0.8897 - val_accuracy: 0.7400
Epoch 4/100
25/25 [==============================] - 0s 8ms/step - loss: 0.1517 - accuracy: 0.9688 - val_loss: 0.9114 - val_accuracy: 0.7400
Epoch 5/100
25/25 [==============================] - 0s 9ms/step - loss: 0.1225 - accuracy: 0.9762 - val_loss: 0.9359 - val_accuracy: 0.7450
Epoch 6/100
25/25 [==============================] - 0s 9ms/step - loss: 0.0986 - accuracy: 0.9850 - val_loss: 0.9709 - val_accuracy: 0.7500
Epoch 7/100
25/25 [==============================] - 0s 9ms/step - loss: 0.0792 - accuracy: 0.9875 - val_loss: 0.9976 - val_accuracy: 0.7400
Epoch 8/100


Epoch 58/100
25/25 [==============================] - 0s 8ms/step - loss: 6.2003e-07 - accuracy: 1.0000 - val_loss: 2.6135 - val_accuracy: 0.7200
Epoch 59/100
25/25 [==============================] - 0s 8ms/step - loss: 5.3286e-07 - accuracy: 1.0000 - val_loss: 2.6219 - val_accuracy: 0.7150
Epoch 60/100
25/25 [==============================] - 0s 9ms/step - loss: 4.6357e-07 - accuracy: 1.0000 - val_loss: 2.6365 - val_accuracy: 0.7150
Epoch 61/100
25/25 [==============================] - 0s 8ms/step - loss: 4.0725e-07 - accuracy: 1.0000 - val_loss: 2.6546 - val_accuracy: 0.7200
Epoch 62/100
25/25 [==============================] - 0s 9ms/step - loss: 3.6254e-07 - accuracy: 1.0000 - val_loss: 2.6631 - val_accuracy: 0.7150
Epoch 63/100
25/25 [==============================] - 0s 9ms/step - loss: 3.2216e-07 - accuracy: 1.0000 - val_loss: 2.6719 - val_accuracy: 0.7150
Epoch 64/100
25/25 [==============================] - 0s 8ms/step - loss: 2.8923e-07 - accuracy: 1.0000 - val_loss: 2.6819 -

In [65]:
sentence = ["trash"]
test = vectorizer.transform(sentence).toarray()

In [66]:
model.predict(test)

array([[4.68093762e-03, 1.24630695e-02, 2.80869249e-02, 9.54252124e-01,
        5.16922097e-04]], dtype=float32)

In [67]:
prediction = model.predict(test).tolist()[0]
score = prediction.index((max(prediction)))
print(score + 1)

4
